In [1]:
import sqlite3
import numpy as np
import pandas as pd

In [2]:
def create_question_response_table() -> tuple:
    csv_file_path = "kaggle_survey_2022_responses.csv"
    first_two_lines = pd.read_csv(csv_file_path, nrows=1)
    temp_df = pd.read_csv(csv_file_path, skiprows=[1], low_memory=False)
    survey_data = temp_df.drop('Duration (in seconds)', axis=1)
    question_indices = [col_name[0] for col_name in survey_data.columns.str.split("_")]
    question_descriptions = first_two_lines.values.ravel()[1:]
    unique_question_indices = pd.Series(question_indices).unique()
    question_descriptions_split = [desc.split(" - ")[0] for desc in question_descriptions]
    unique_question_descriptions = pd.Series(question_descriptions_split).unique()
    question_types = ["Multiple Response" if "(Select all that apply)" in desc else "Multiple Choice" for desc in unique_question_descriptions]
    question_table = pd.DataFrame()
    question_table["id"] = np.arange(1, unique_question_indices.size + 1)
    question_table["question_index"] = unique_question_indices
    question_table["type"] = question_types
    question_table["description"] = unique_question_descriptions
    n_rows = survey_data.shape[0]
    survey_data.insert(0, "id", np.arange(1, n_rows + 1))
    response_table = pd.melt(survey_data, id_vars="id", var_name="question_index", value_name="response").dropna()
    question_indices = [qi[0] for qi in response_table["question_index"].str.split("_")]
    response_table["question_index"] = question_indices
    merge_response_question_table = pd.merge(response_table, question_table[["id", "question_index"]], left_on="question_index", right_on="question_index")
    response_table = merge_response_question_table[["id_x", "id_y", "response"]]
    response_table.columns = ["respondant_id", "question_id", "choice"]
    return question_table, response_table

In [3]:
questions, responses = create_question_response_table()

In [4]:
questions

,id,question_index,type,description
0,1,Q2,Multiple Choice,What is your age (# years)?
1,2,Q3,Multiple Choice,What is your gender?
2,3,Q4,Multiple Choice,In which country do you currently reside?
3,4,Q5,Multiple Choice,"Are you currently a student? (high school, uni..."
4,5,Q6,Multiple Response,On which platforms have you begun or completed...
5,6,Q7,Multiple Response,What products or platforms did you find to be ...
6,7,Q8,Multiple Choice,What is the highest level of formal education ...
7,8,Q9,Multiple Choice,Have you ever published any academic research ...
8,9,Q10,Multiple Choice,Did your research make use of machine learning?
9,10,Q11,Multiple Choice,For how many years have you been writing code ...


In [5]:
responses

,respondant_id,question_id,choice
0,1,1,30-34
1,2,1,30-34
2,3,1,18-21
3,4,1,55-59
4,5,1,45-49
...,...,...,...
834427,23815,43,Other
834428,23839,43,Other
834429,23987,43,Other
834430,23989,43,Other


In [6]:
con = sqlite3.connect('KaggleSurvey2022.db')
questions.to_sql('questions', con, if_exists='replace', index=False)
responses.to_sql('responses', con, if_exists='replace', index=False)
cur = con.cursor()

In [7]:
create_questions = """
PRAGMA foreign_keys=off;
BEGIN TRANSACTION;
ALTER TABLE questions RENAME TO questions_no_keys;
CREATE TABLE questions (
    id INTEGER,
    question_index INTEGER,
    type TEXT,
    description TEXT,
    PRIMARY KEY (id)
);
INSERT INTO questions SELECT * FROM questions_no_keys;
COMMIT;
PRAGMA foreign_keys=on;
"""
cur.executescript(create_questions)
con.commit()

In [8]:
create_responses = """
PRAGMA foreign_keys=off;
BEGIN TRANSACTION;
ALTER TABLE responses RENAME TO responses_no_keys;
CREATE TABLE responses (
    respondant_id INTEGER,
    question_id INTEGER,
    choice TEXT,
    FOREIGN KEY (question_id) REFERENCES questions (id) 
            ON DELETE CASCADE ON UPDATE NO ACTION
);
INSERT INTO responses SELECT * FROM responses_no_keys;
COMMIT;
PRAGMA foreign_keys=on;
"""
cur.executescript(create_responses)
con.commit()

In [9]:
drop_tables = """
DROP TABLE questions_no_keys;
DROP TABLE responses_no_keys;
"""
cur.executescript(drop_tables)
con.commit()

In [10]:
con.close()